In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit - Sequential workflow

### Install adk

In [1]:
%pip install --upgrade --quiet google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00


In [2]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[2]
print(MODEL_NAME)

gemini-2.5-flash-preview-04-17


In [11]:
# @title Authentication to access to GCP
import sys
from pprint import pprint
from IPython.display import display, Markdown

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Initialize Vertex AI

* Please refer to configuration to **initialize Vertex AI : [Link](https://google.github.io/adk-docs/get-started/quickstart/#set-up-the-model)**
* You can set up api key rather than usning Vertex AI.


In [4]:
import os

GOOGLE_GENAI_USE_VERTEXAI = "TRUE"                            #@param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = GOOGLE_GENAI_USE_VERTEXAI

GOOGLE_CLOUD_PROJECT = "ai-hangsik"                           #@param {type:"string"}
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

GOOGLE_CLOUD_LOCATION = "us-central1"                         #@param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"] = GOOGLE_CLOUD_LOCATION




## adk with tools

In [5]:
import asyncio

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.adk.tools import google_search
from google.adk.agents import SequentialAgent
from google.adk.agents import ParallelAgent


In [6]:
positive_agent = LlmAgent(
    name="PositiveAgent",
    model=MODEL_NAME,
    instruction="""You are a helpful AI assistant to find positive side of given technology
                   Add the title "## Positive aspects" to the response.

    """,
    description="Find positive aspects for the given technology",
    output_key="positive_perspectives",
    tools=[google_search],
)

negative_agent = LlmAgent(
    name="NegativeAgent",
    model=MODEL_NAME,
    instruction="""You are a helpful AI assistant to find negative side of given technology
                       Add the title "## Negative aspects" to the response.

    """,
    description="Find negative aspects for the given technology",
    output_key="negative_perspectives",
    tools=[google_search],

)

conculusion_agent = LlmAgent(
    name="ConclusionAgent",
    model=MODEL_NAME,
    instruction="""You are a helpful AI assistant to draw conclusion by reasoning the both side of positive and negative
                   Add the title "## Conclusion" to the response.


    """,
    description="Draw conculusion.",
    output_key="conculusion",
    tools=[google_search],
)

review_agent = LlmAgent(
    name="ReviewAgent",
    model=MODEL_NAME,
    instruction="""You are a helpful AI assistant to review the conclusion
                   Add the title "## Review Conclusion" to the response.

    """,
    description="Review conculusion.",
    output_key="review",
    tools=[google_search],
)

In [7]:
parallel_research_agent = ParallelAgent(
    name="ParallelResearchAgent",
    sub_agents=[positive_agent, negative_agent],
    description="Runs multiple research agents in parallel to gather information."
)

In [8]:

pipeline_agent = SequentialAgent(
    name="PipelineAgent",
    sub_agents=[parallel_research_agent, conculusion_agent, review_agent],
    description="Executes a sequence of positive_agent, negative_agent, and refactoring.",
)


In [14]:
app_name = "Assistant"
user_id = "shins"

# query = "what is Generative AI?"
query = "생성형 AI 가 뭔가요 ?"

session_service = InMemorySessionService()
session = session_service.create_session(app_name=app_name, user_id=user_id)

runner = Runner(agent=pipeline_agent,
                app_name=session.app_name,
                session_service=session_service)

content = types.Content(role='user', parts=[types.Part(text=query)])

events = runner.run(user_id=session.user_id,
                    session_id=session.id,
                    new_message=content)

for event in events:
    # print(event.content.parts[0].text)
    display(Markdown(event.content.parts[0].text))

    # if event.is_final_response():
    #     final_response = event.content.parts[0].text
    #     print(f"Agent Response: {final_response}\n")
    # else:
    #     print("No results")

생성형 AI는 사용자의 요청(프롬프트)에 맞춰 텍스트, 이미지, 오디오, 비디오 등 새롭고 독창적인 콘텐츠를 만들어내는 인공지능 시스템입니다 [1, 2, 5, 7]. 기존의 인공지능이 데이터를 분석하고 분류하는 데 주력했다면, 생성형 AI는 학습한 데이터를 기반으로 새로운 결과물을 창조한다는 점이 다릅니다 [1, 9].

생성형 AI는 대규모 데이터를 학습하여 데이터의 패턴과 구조를 익히고, 이를 바탕으로 유사한 특징을 가진 새로운 데이터를 생성합니다 [6, 7]. 예를 들어, 텍스트 생성 모델은 문장의 앞부분을 학습하여 뒷부분을 예측하는 방식으로 훈련되며, 훈련이 끝나면 스스로 새로운 텍스트를 생성할 수 있습니다 [1].

생성형 AI는 딥러닝 기술을 기반으로 하며, 인공 신경망과 같은 복잡한 아키텍처를 활용하여 정보를 처리하고 새로운 콘텐츠를 생성합니다 [5]. 생성형 AI 모델에는 GAN (생성형 적대 신경망), VAE (변이형 오토인코더), 트랜스포머 기반 모델 등이 있습니다 [2, 5, 6].

ChatGPT, Bard(Gemini로 변경), Stable Diffusion, Midjourney, DALL-E 등이 대표적인 생성형 AI 시스템으로 알려져 있습니다 [6].

## Positive aspects

생성형 AI의 긍정적인 측면은 다음과 같습니다.

*   **업무 자동화 및 효율성 향상:** 반복적이고 지루한 작업을 자동화하여 업무 효율성을 크게 높일 수 있습니다 [1, 4, 12]. 기업의 판매, 일반, 관리 비용을 절감하는 데 기여할 수 있으며, 복잡한 작업 자동화 및 수동 작업량 감소를 통해 비즈니스 프로세스 관리를 가속화합니다 [10].
*   **창의적인 작업 지원 및 아이디어 제공:** 디자인, 예술, 음악 등 창의적인 분야에서 아이디어를 얻거나 결과물을 만드는 데 도움을 받을 수 있습니다 [1, 3]. 인간의 창조력을 넘어서는 결과물을 만들어낼 수도 있습니다 [3]. 복잡한 이미지나 영상 제작과 같이 인간에게 시간이 오래 걸리는 작업을 더 빠르고 효율적으로 수행하도록 돕습니다 [9].
*   **고객 경험 향상:** 고객 데이터를 분석하여 개인 맞춤형 경험을 제공하고, 개인화된 광고 카피 작성이나 마케팅 캠페인 자동화를 통해 광고 효과를 극대화할 수 있습니다 [1]. 사람과 유사한 대화 에이전트를 통해 고객 서비스를 개선하고, 개인화된 영업 보조원을 통해 고객 관계를 재창조할 수 있습니다 [2]. 사용자의 취향과 관심사에 맞는 개인화된 콘텐츠를 생성하여 사용자 경험을 향상시킬 수 있습니다 [4, 9].
*   **정보 검색 및 활용:** 문서 요약, 번역 등 언어 기반 작업에서 뛰어난 성능을 발휘하여 중요한 정보를 빠르게 파악하고 커뮤니케이션을 개선할 수 있습니다 [1]. 대규모 언어 모델을 통해 질문에 답하고, 콘텐츠를 생성하며, 언어를 번역하여 정보 검색을 효율적이고 개인화할 수 있습니다 [10].
*   **새로운 분야 연구 및 탐구 지원:** 인간의 한계를 극복하고 의학, 과학, 예술 등 다양한 분야에서의 연구와 탐구를 도울 수 있습니다 [4]. 질병의 조기 진단과 맞춤형 치료를 가능하게 하고, 새로운 약물 및 치료법 개발을 촉진할 수 있습니다 [11].
*   **학습 및 발전:** 대규모 데이터 학습을 통해 지속적으로 발전하고 향상되며, 새로운 정보를 받아들이고 적응하여 더욱 정교한 결과물을 생성합니다 [4]. AI 기반 학습 프로그램을 통해 학생들의 학업 성과를 향상시키는 사례도 있습니다 [10].
*   **다양한 콘텐츠 생성 용이:** 문서, 이미지, 사운드, 영상 등 다양한 형태의 콘텐츠를 손쉽게 생성할 수 있습니다 [8]. 전문 지식이 없는 일반인도 쉽게 활용하여 텍스트, 이미지, 음성, 3D 모델까지 생성할 수 있습니다 [11].

생성형 AI는 사용자의 요청에 따라 텍스트, 이미지, 영상, 음악 등 새롭고 독창적인 콘텐츠나 아이디어를 생성하는 인공지능의 한 분야입니다 [1, 4, 5, 6, 8]. 대량의 데이터를 학습하여 데이터의 구조와 특징을 모방하고, 이를 바탕으로 새로운 결과물을 만들어냅니다 [1, 5, 9]. 기존의 인공지능이 데이터 분석, 얼굴 인식, 음성 인식 등 주어진 데이터를 처리하는 데 초점을 맞췄다면, 생성형 AI는 더 나아가 학습한 내용을 기반으로 새로운 것을 창조하는 데 강점이 있습니다 [1].

생성형 AI는 머신러닝 모델, 특히 딥러닝의 원리에 따라 작동하며, 인공 신경망이라는 복잡한 구조를 사용합니다 [5]. 정보를 처리하고 새로운 데이터를 생성하는 과정에서 생성자와 판별자라는 두 개의 신경망이 경쟁적으로 학습하는 생성적 적대 신경망(GAN) 모델 등이 활용됩니다 [1, 4, 6]. 또한 대규모 언어 모델(LLM)이나 트랜스포머 기반 모델은 텍스트 생성 분야에서 큰 발전을 이루었습니다 [5, 8].

생성형 AI는 다양한 분야에서 활용되고 있습니다. 예를 들어, 고객 서비스를 위한 대화형 에이전트 개발, 마케팅 콘텐츠 제작, 제품 개발 및 설계, 번역 및 요약과 같은 언어 기반 작업, 예술 작품 생성 등에 사용될 수 있습니다 [1, 4, 7, 8]. 또한 데이터 분석 및 예측, 합성 데이터 생성 등에도 활용됩니다 [4, 15].

## Negative aspects

생성형 AI는 여러 장점에도 불구하고 몇 가지 문제점과 한계를 가지고 있습니다 [2, 7, 13, 17].

*   **편향된 데이터와 무의미한 답변:** 학습 데이터의 편향이 결과물에 반영되어 편향된 내용을 생성하거나, 학습된 데이터 및 패턴의 제약으로 인해 무의미하거나 상식에서 벗어난 답변을 반복할 수 있습니다 [2, 11].
*   **높은 학습 및 유지보수 비용:** 생성형 AI 모델을 훈련하고 유지하는 데 막대한 계산 자원과 비용이 필요하며, 이는 소규모 조직에게는 부담이 될 수 있습니다 [2, 5].
*   **거짓 정보 및 환각 문제:** 잘못된 정보나 사실이 아닌 내용을 마치 사실인 것처럼 생성하는 '환각(Hallucination)' 문제가 지속적으로 발생합니다 [2, 11].
*   **저작권 및 지식재산권 침해:** 인터넷상의 방대한 데이터를 학습하는 과정에서 원저작자의 허가 없이 콘텐츠를 사용하여 저작권이나 지식재산권을 침해할 수 있으며, 이로 인해 법적 분쟁이 발생하기도 합니다 [2, 3, 7, 10, 17]. 생성된 콘텐츠의 저작권 문제 또한 모호한 부분이 있습니다 [3, 12].
*   **악용 가능성 (딥페이크 등):** 생성형 AI 기술을 사용하여 가짜 뉴스나 정교한 딥페이크 영상 등을 만들어 범죄에 악용될 수 있습니다 [1, 7, 10, 14]. 음성 인증 시스템을 통과하는 AI 복제 음성 사례도 보고되고 있습니다 [3].
*   **정보의 정확성과 신뢰성 문제:** AI가 생성한 정보의 정확성과 신뢰성이 항상 보장되지 않아 사용자들에게 혼란을 줄 수 있습니다 [7, 12].
*   **인간의 창의성 및 윤리적 문제:** 인간의 창의성을 대체하거나 창작 동기를 저하시킬 수 있다는 우려가 있으며, AI 생성 콘텐츠와 관련된 윤리적인 문제들이 제기됩니다 [3, 12]. 특히 의료/건강 분야에서는 '가짜 의사' 이미지를 이용한 허위 광고나 AI가 생성한 가짜 처방전과 같은 오남용 사례도 나타나고 있습니다 [16].
*   **정적 정보:** 학습된 데이터 내에서 정보를 생성하므로 최신 정보나 실시간으로 변화하는 정보를 반영하는 데 한계가 있을 수 있습니다 [2].
*   **고급 기능 부족 및 제한적인 협업 기능:** 아이디어 초안 수준에는 적합하지만, 정교하거나 복잡한 디자인 등 고급 기능을 구현하는 데는 한계가 있을 수 있으며, 협업 도구로서의 기능도 제한적일 수 있습니다 [13].
*   **완전 자동화의 한계:** 모든 종류의 작업을 완전히 자동화하기는 어려우며, 특히 창의적이고 복잡한 작업의 경우 인간의 검토와 수정이 필요하여 추가적인 비용과 시간이 발생할 수 있습니다 [15].

생성형 AI는 사용자의 요청에 따라 텍스트, 이미지, 오디오, 비디오 등 새롭고 독창적인 콘텐츠를 만들어내는 인공지능 시스템입니다. 기존 인공지능이 데이터 분석 및 분류에 중점을 두었다면, 생성형 AI는 학습된 대규모 데이터를 기반으로 새로운 결과물을 창조한다는 특징을 가집니다. 딥러닝 기술과 인공 신경망을 활용하며, GAN, VAE, 트랜스포머 기반 모델 등이 있습니다. ChatGPT, Gemini, Stable Diffusion 등이 대표적인 예시입니다.

## Conclusion

생성형 AI는 업무 자동화 및 효율성 향상, 창의적인 작업 지원, 고객 경험 개선, 정보 검색 및 활용 용이, 새로운 분야 연구 지원 등 다양한 긍정적인 측면을 가지고 있습니다. 반복적인 작업을 자동화하고 창의적인 아이디어를 제공하며 개인 맞춤형 콘텐츠를 생성하여 생산성과 만족도를 높일 수 있습니다. 의료, 과학, 예술 등 여러 분야에서 연구 및 탐구를 돕고, 다양한 형태의 콘텐츠를 쉽게 만들어낼 수 있습니다.

하지만 생성형 AI는 학습 데이터의 편향으로 인한 결과물 편향, 높은 학습 및 유지보수 비용, 잘못된 정보를 사실처럼 생성하는 환각 문제, 저작권 및 지식재산권 침해 가능성, 딥페이크와 같은 악용 가능성, 정보의 정확성과 신뢰성 문제, 인간의 창의성 저해 및 윤리적 문제, 최신 정보 반영의 한계, 고급 기능 부족 및 제한적인 협업 기능, 완전 자동화의 한계 등 여러 문제점과 한계도 가지고 있습니다. 학습 데이터의 편향은 결과물의 신뢰도를 떨어뜨리고, 높은 비용은 접근성을 제한할 수 있습니다. 저작권 문제나 악용 가능성은 법적, 윤리적 문제를 야기하며, 정보의 정확성 문제는 사용자의 혼란을 초래할 수 있습니다.

결론적으로 생성형 AI는 혁신적인 기술로서 다양한 분야에 긍정적인 영향을 미칠 잠재력이 크지만, 동시에 해결해야 할 여러 문제점과 윤리적 고려 사항을 안고 있습니다. 기술의 발전과 함께 이러한 문제점들을 해결하기 위한 지속적인 연구와 사회적 논의가 필요하며, 사용자는 생성된 정보의 한계를 인지하고 비판적으로 활용하는 자세가 중요합니다.

## Review Conclusion

제공된 결론은 생성형 AI의 정의, 긍정적 측면 및 부정적 측면을 잘 요약하고 있습니다. 생성형 AI가 가져올 수 있는 잠재력과 함께 해결해야 할 과제들을 명확하게 제시하고 있으며, 기술 발전과 더불어 사용자들의 비판적인 활용 자세의 중요성을 강조하며 마무리하고 있습니다.

생성형 AI는 사용자의 요청에 따라 텍스트, 이미지, 오디오, 비디오 등 새롭고 독창적인 콘텐츠를 만들어내는 인공지능 시스템입니다. 기존 인공지능이 데이터 분석 및 분류에 중점을 두었다면, 생성형 AI는 학습된 대규모 데이터를 기반으로 새로운 결과물을 창조한다는 특징을 가집니다. 딥러닝 기술과 인공 신경망을 활용하며, GAN, VAE, 트랜스포머 기반 모델 등이 있습니다. ChatGPT, Gemini, Stable Diffusion 등이 대표적인 예시입니다.

## Conclusion

생성형 AI는 업무 자동화 및 효율성 향상, 창의적인 작업 지원, 고객 경험 개선, 정보 검색 및 활용 용이, 새로운 분야 연구 지원 등 다양한 긍정적인 측면을 가지고 있습니다. 반복적인 작업을 자동화하고 창의적인 아이디어를 제공하며 개인 맞춤형 콘텐츠를 생성하여 생산성과 만족도를 높일 수 있습니다. 의료, 과학, 예술 등 여러 분야에서 연구 및 탐구를 돕고, 다양한 형태의 콘텐츠를 쉽게 만들어낼 수 있습니다.

하지만 생성형 AI는 학습 데이터의 편향으로 인한 결과물 편향, 높은 학습 및 유지보수 비용, 잘못된 정보를 사실처럼 생성하는 환각 문제, 저작권 및 지식재산권 침해 가능성, 딥페이크와 같은 악용 가능성, 정보의 정확성과 신뢰성 문제, 인간의 창의성 저해 및 윤리적 문제, 최신 정보 반영의 한계, 고급 기능 부족 및 제한적인 협업 기능, 완전 자동화의 한계 등 여러 문제점과 한계도 가지고 있습니다. 학습 데이터의 편향은 결과물의 신뢰도를 떨어뜨리고, 높은 비용은 접근성을 제한할 수 있습니다. 저작권 문제나 악용 가능성은 법적, 윤리적 문제를 야기하며, 정보의 정확성 문제는 사용자의 혼란을 초래할 수 있습니다.

결론적으로 생성형 AI는 혁신적인 기술로서 다양한 분야에 긍정적인 영향을 미칠 잠재력이 크지만, 동시에 해결해야 할 여러 문제점과 윤리적 고려 사항을 안고 있습니다. 기술의 발전과 함께 이러한 문제점들을 해결하기 위한 지속적인 연구와 사회적 논의가 필요하며, 사용자는 생성된 정보의 한계를 인지하고 비판적으로 활용하는 자세가 중요합니다.

## End of notebook